## Search for similar expressions

In [13]:
import numpy as np
import cv2 as cv
import glob
import base64
from sklearn.cluster import KMeans
import pandas as pd
import joblib

SIFT

In [3]:
deses = list()
folder_dir = "C:/Users/andre/Downloads/train2017/*.jpg"
for i in glob.glob(folder_dir):
    img = cv.imread(i)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp, des = sift.detectAndCompute(gray,None)
    deses.append(des)

train = np.vstack(deses)
train.shape

(186062, 128)

Обучаем модель

In [4]:
model = KMeans(n_clusters=512)
model.fit(train)

KMeans(n_clusters=512)

Сохраняем модель

In [5]:
#save = pickle.dumps(model)
joblib.dump(model, 'model.sav')

['model.sav']

Используем сохранённую модель для получения множества классов

In [6]:
#load = pickle.loads(save)
load = joblib.load('model.sav')

In [14]:
path = list()
embeddings = list()
for i in glob.glob('C:/Users/andre/Downloads/JPEGImages/*.jpg'):
    img = cv.imread(i)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    sift = cv.SIFT_create()
    kp, des = sift.detectAndCompute(gray,None)
    if des is not None: 
        path.append(i)
        embeddings.append(base64.b64encode(load.predict(des)))

Создаём таблицу

In [16]:
df = pd.DataFrame(data={'path': path,'embedding': embeddings})
# Исправляем путь
df['path'] = df['path'].str.replace('\\', '/')
df.to_csv('data.csv', index=False)

C:\Users\andre\AppData\Local\Temp\ipykernel_20724\1778191709.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['path'] = df['path'].str.replace('\\', '/')


Итог

In [17]:
data = pd.read_csv("data.csv", delimiter=',')
data['embedding'] = data['embedding'].apply(lambda x: np.frombuffer(base64.b64decode(bytes(x[2:-1],encoding='ascii')), dtype=np.int32))
data

,path,embedding
0,C:/Users/andre/Downloads/JPEGImages/2007_00002...,"[372, 341, 323, 453, 429, 403, 156, 50, 2, 324..."
1,C:/Users/andre/Downloads/JPEGImages/2007_00003...,"[309, 346, 298, 481, 37, 189, 298, 148, 29, 18..."
2,C:/Users/andre/Downloads/JPEGImages/2007_00003...,"[378, 323, 407, 329, 494, 212, 397, 111, 341, ..."
3,C:/Users/andre/Downloads/JPEGImages/2007_00003...,"[171, 88, 424, 373, 373, 373, 258, 259, 258, 2..."
4,C:/Users/andre/Downloads/JPEGImages/2007_00004...,"[359, 478, 281, 492, 241, 10, 10, 391, 324, 39..."
...,...,...
17119,C:/Users/andre/Downloads/JPEGImages/2012_00432...,"[324, 67, 251, 479, 458, 373, 479, 125, 409, 4..."
17120,C:/Users/andre/Downloads/JPEGImages/2012_00432...,"[266, 387, 383, 12, 205, 322, 35, 293, 329, 44..."
17121,C:/Users/andre/Downloads/JPEGImages/2012_00432...,"[372, 387, 353, 183, 33, 56, 84, 42, 42, 327, ..."
17122,C:/Users/andre/Downloads/JPEGImages/2012_00433...,"[218, 50, 65, 447, 261, 54, 363, 143, 439, 511..."


Расчет количества вхождения в каждый кластер

In [18]:
matrix=np.zeros((len(data),512),dtype=np.uint8)
for j in range (len(data)):
    for i in range (len(data['embedding'][j])):
        matrix[j,data['embedding'][j][i]] += 1
    data['embedding'][j] = (base64.b64encode(matrix[j]))
data.to_csv('data1.csv',index=False) 
data['embedding']

0        b'BAICAwQCBgIFAAEGBAIGBQMEAwMIBQQBBQYCAAAHAgAC...
1        b'AAACAAMAAAIAAAgAAAAEAAAAAQAAAQAEAQAAEQADAgIA...
2        b'AAAAAQAAAQAAAgEAAAIAAAEAAQEAAAEBAQEBAQEBAQAB...
3        b'AgABAAIBAAAAAQUABgADAAAAAAABAAABAAABAAICAQAB...
4        b'AgEKAAAAAQEAABkAAQACAAEAAAAAAwABAgAAAQADAgMB...
                               ...                        
17119    b'AgEAAgIAAAEBBwAAAgADAQAAAAEDAgMAAAEFAAUDAAEA...
17120    b'AgQAAgIBAgIHBQAABwMDAwAAAAIBAwEFBgABAAYBAAED...
17121    b'AwQAAQEBAAICBAQAAgIBAQAAAAICAAIAAwECAQMFAwAB...
17122    b'AgEEAAgBAQECCAEBAQEAAAABAgEDAgEAAAEEAAUAAgEG...
17123    b'BAIABAMAAAAAAQIABQEBAgIAAAADAAIAAAAAAQECAAAE...
Name: embedding, Length: 17124, dtype: object